# Registration transform example

This example illustrates how to use a RegistrationTransform to temporally align the frames of an EOPatch using different algorithms.

## Create cloudless timelapse

In [1]:
import datetime
import os
import warnings

import imageio
import numpy as np

from eolearn.core import FeatureType
from eolearn.core.exceptions import EORuntimeWarning
from eolearn.coregistration import ECCRegistrationTask, PointBasedRegistrationTask
from eolearn.coregistration.extra.thunder import ThunderRegistrationTask
from eolearn.features import SimpleFilterTask
from eolearn.io import SentinelHubInputTask
from sentinelhub import CRS, BBox, DataCollection

In [2]:
# The code raises warnings when automatically adjusting problematic data, which we ignore in the notebook for clarity.
warnings.filterwarnings(action="ignore", category=RuntimeWarning)
warnings.filterwarnings(action="ignore", category=EORuntimeWarning)
warnings.filterwarnings(action="ignore", category=np.VisibleDeprecationWarning)

### Set up BBox of ROI and time interval

In [3]:
roi_bbox = BBox(bbox=[31.112895, 29.957240, 31.154222, 29.987687], crs=CRS.WGS84)
time_interval = ("2018-01-01", "2020-06-01")

feat_bands = (FeatureType.DATA, "BANDS")

This predicate function filters the images with a cloud coverage larger than a threshold to ensure images do not contain cloudy pixels

In [4]:
class MaxCloudCoveragePredicate:
    def __init__(self, max_cloud_coverage: float):
        self.max_cloud_coverage = max_cloud_coverage

    def __call__(self, cloud_mask: np.ndarray):
        width, height, _ = cloud_mask.shape
        cloud_coverage = np.sum(cloud_mask) / (width * height)
        return cloud_coverage <= self.max_cloud_coverage

Tasks:
 * download S2 images (bands needed for true color visualization)
 * download cloud mask (CLM) provided by Sentinel Hub
 * filter out images with cloud coverage larger than a given threshold

In [5]:
download_task = SentinelHubInputTask(
    data_collection=DataCollection.SENTINEL2_L1C,
    bands_feature=feat_bands,
    resolution=10,
    maxcc=0.5,
    bands=["B02", "B03", "B04"],
    time_difference=datetime.timedelta(hours=2),
    additional_data=[(FeatureType.MASK, "dataMask", "IS_DATA"), (FeatureType.MASK, "CLM")],
)
filter_clouds = SimpleFilterTask((FeatureType.MASK, "CLM"), MaxCloudCoveragePredicate(max_cloud_coverage=0.05))

Execute timelapse as chain of transforms and get result as an eopatch

In [6]:
eopatch = download_task.execute(bbox=roi_bbox, time_interval=time_interval)

eopatch_clean = filter_clouds.execute(eopatch)

eopatch_clean

EOPatch(
  data={
    BANDS: numpy.ndarray(shape=(130, 331, 404, 3), dtype=float32)
  }
  mask={
    CLM: numpy.ndarray(shape=(130, 331, 404, 1), dtype=uint8)
    IS_DATA: numpy.ndarray(shape=(130, 331, 404, 1), dtype=bool)
  }
  meta_info={
    maxcc: 0.5
    size_x: 404
    size_y: 331
    time_difference: 7200.0
    time_interval: ('2018-01-01T00:00:00', '2020-06-01T23:59:59')
  }
  bbox=BBox(((31.112895, 29.95724), (31.154222, 29.987687)), crs=CRS('4326'))
  timestamp=[datetime.datetime(2018, 1, 4, 8, 33, 28), ..., datetime.datetime(2020, 5, 28, 8, 41, 58)], length=130
)

### Help function to create GIFs
The function uses a `rescale_factor` for a nicer picture. We set it to 2.8 by default.

In [7]:
def make_gif(bands: np.ndarray, project_dir: str, filename: str, fps: int, rescale_factor: float = 2.8):
    # Generates a GIF animation from an EOPatch.
    with imageio.get_writer(os.path.join(project_dir, filename), mode="I", fps=fps) as writer:
        for image in bands:
            writer.append_data(np.array(np.clip(rescale_factor * image[..., [2, 1, 0]], 0, 255), dtype=np.uint8))

Write clean EOPatch to GIF

In [8]:
make_gif(bands=eopatch_clean[feat_bands] * 255, project_dir=".", filename="eopatch_clean.gif", fps=3)

## Run registrations

The constructor of the Registration objects takes the attribute type, field name and index of the channel to be used for registration, a dictionary specifying the parameters of the registration, and the interpolation method to be applied to the images. The interpolation methods are (NEAREST, LINEAR and CUBIC). Default is CUBIC. A nearest neighbour interpolation is used on ground-truth data to avoid creation of new labels.


### Thunder registration

This algorithm computes translations only between pairs of images, using correlation on the Fourier transforms of the images

In [9]:
coregister_thunder = ThunderRegistrationTask(feat_bands, channel=2)

eopatch_thunder = coregister_thunder(eopatch_clean)

make_gif(bands=eopatch_thunder[feat_bands] * 255, project_dir=".", filename="eopatch_thunder.gif", fps=3)

### Enhanced Cross-Correlation in OpenCV

This algorithm uses intensity values to maximise cross-correlation between pair of images. It uses an Euler transformation (x,y translation plus rotation).

In [10]:
params = {"MaxIters": 200}

coregister_ecc = ECCRegistrationTask(feat_bands, channel=2, params=params)

eopatch_ecc = coregister_ecc(eopatch_clean)

make_gif(bands=eopatch_ecc[feat_bands] * 255, project_dir=".", filename="eopatch_ecc.gif", fps=3)

### Point-Based Registration in OpenCV

Three transformation models are supported for point-based registration, i.e. Euler, PartialAffine and Homography. These methods compute feature descriptors (i.e. SIFT or SURF) of the pair of images to be registered, and estimate a robust transformation using RANSAC to align the matching points. These methods perform poorly compared to the other methods due to the inaccuracies of the feature extraction, point-matching and model fitting. If unplausible transformations are estimated, a warning is issued and an identity matrix is employed instead of the estimated transform. Default parameters are (Model=Euler, Descriptor=SIFT, RANSACThreshold=7.0, MaxIters=1000).

In [11]:
params = {"Model": "Euler", "Descriptor": "SURF", "RANSACThreshold": 7.0, "MaxIters": 1000}

coregister_pbased = PointBasedRegistrationTask(feat_bands, channel=2, params=params)

eopatch_pbased = coregister_pbased(eopatch_clean)

make_gif(bands=eopatch_pbased[feat_bands] * 255, project_dir=".", filename="eopatch_pbased.gif", fps=3)